In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing
import urllib

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
curWeek = 16

In [3]:
P5 = ['ACC','Pac-12','Big Ten','Big 12','SEC','Notre Dame']
G5 = ['Conference USA','Sun Belt','Mid-American','American Athletic','Mountain West','FBS Independents']

def seasonProgression(dfR):
    totalGames = len(dfR)
    curWeek = int(dfR['week']) + 1
    return curWeek/(2*totalGames)

def winningTeam(dfR):
    if dfR['home_points'] > dfR['away_points']:
        return dfR['home_team']
    else:
        return dfR['away_team']
    
def teamWin(dfR, team):
    if dfR.winner == team:
        return True
    else:
        return False
    
def findOpp(dfR):
    if dfR.home == True:
        return dfR.away_team
    else:
        return dfR.home_team
def findOppConf(dfR):
    if dfR.home == True:
        return dfR.away_conference
    else:
        return dfR.home_conference
    
def locMult(dfR):
    if dfR['neutral_site'] == True:
        return 1.05
    elif dfR['home'] == True:
        return 1.1
    else:
        return 1.
    
def pointDiff(dfR,team):
    if dfR.home_team == team:
        return dfR.home_points - dfR.away_points
    else:
        return dfR.away_points - dfR.home_points
    
def confMult(dfR):
    if dfR.opp_conference in P5:
        return 1.
    elif dfR.opp_conference in G5:
        return 0.5
    else:
        return 0.15
    
def winQ(dfR,week): 
    if dfR.opp_conference == None:
        oppWinPct = 0.5
    else:
#         print(dfR.opp_conference)
        oppWinPct = FBSdict[dfR.opp].loc[FBSdict[dfR.opp].week <= week, 'winPct'].tail(1).item()
    
    if dfR.teamWin == True:
        winMult = 1
        oppFactor = oppWinPct
    else:
        winMult = -1
        oppFactor = 1 - oppWinPct
    
    return oppFactor * dfR.seasonProg * dfR.conf_mult * winMult*(dfR.loc_mult + 0.01*dfR.point_diff)

def calcPct(team, week):
    winqweek = 'win_q_'+str(week)
    wins = FBSdict[team].loc[FBSdict[team].week <= week, 'wins'].tail(1).item()
    losses = FBSdict[team].loc[FBSdict[team].week <= week, 'losses'].tail(1).item()
#     print(winqweek,wins,losses,FBSdict[team].loc[FBSdict[team].week <= week, winqweek].mean())
    return 0.9 * FBSdict[team].loc[FBSdict[team].week <= week, winqweek].mean() + 0.01*(wins-losses)

In [4]:
json_url = urllib.request.urlopen('https://api.collegefootballdata.com/games?year=2019&seasonType=regular')
jsonFile = json.loads(json_url.read())
curSch = pd.DataFrame(jsonFile)

In [5]:
json_url2 = urllib.request.urlopen('https://api.collegefootballdata.com/games?year=2019&seasonType=postseason')
jsonFile2 = json.loads(json_url2.read())
postSch = pd.DataFrame(jsonFile2)
postSch['week'] = postSch['week'] + 15
curSch = curSch.append(postSch)
curSch = curSch.dropna(subset=['away_points'])

In [6]:
curSch.loc[curSch.away_team == 'Notre Dame','away_conference'] = 'Notre Dame'
curSch.loc[curSch.home_team == 'Notre Dame','home_conference'] = 'Notre Dame'

In [7]:
curSch.tail(10)

,id,season,week,season_type,start_date,neutral_site,conference_game,attendance,venue_id,venue,home_team,home_conference,home_points,home_line_scores,home_post_win_prob,away_team,away_conference,away_points,away_line_scores,away_post_win_prob
28,401147695,2019,16,postseason,2019-12-31T21:30:00.000Z,True,False,None,3619,Arizona Stadium,Georgia State,Sun Belt,17.0,"[7, 3, 7, 0]",0.016546421618826083,Wyoming,Mountain West,38.0,"[17, 7, 14, 0]",0.9834535783811739
29,401135285,2019,16,postseason,2020-01-01T00:30:00.000Z,True,False,None,3604,Alamodome,Texas,Big 12,38.0,"[3, 7, 14, 14]",0.9983780419333973,Utah,Pac-12,10.0,"[0, 0, 3, 7]",0.0016219580666027023
30,401135287,2019,16,postseason,2020-01-01T18:00:00.000Z,True,False,None,3886,Raymond James Stadium,Auburn,SEC,24.0,"[10, 7, 7, 0]",0.07649296797330014,Minnesota,Big Ten,31.0,"[10, 14, 0, 7]",0.9235070320266998
31,401135286,2019,16,postseason,2020-01-01T18:00:00.000Z,True,False,None,4013,Camping World Stadium,Alabama,SEC,35.0,"[7, 7, 7, 14]",0.9984405285143154,Michigan,Big Ten,16.0,"[10, 6, 0, 0]",0.0015594714856845826
32,401135288,2019,16,postseason,2020-01-01T22:10:00.000Z,True,False,None,1056,Rose Bowl,Wisconsin,Big Ten,27.0,"[10, 7, 7, 3]",0.6306454084306707,Oregon,Pac-12,28.0,"[7, 7, 7, 7]",0.36935459156932926
33,401135289,2019,16,postseason,2020-01-02T01:45:00.000Z,True,False,None,3493,Mercedes-Benz Superdome,Baylor,Big 12,14.0,"[0, 0, 14, 0]",0.004284879399672174,Georgia,SEC,26.0,"[3, 16, 7, 0]",0.9957151206003279
34,401135290,2019,16,postseason,2020-01-02T20:00:00.000Z,True,False,None,3803,Legion Field,Cincinnati,American Athletic,38.0,"[7, 10, 7, 14]",0.9973304008616461,Boston College,ACC,6.0,"[0, 0, 6, 0]",0.002669599138353851
35,401135291,2019,16,postseason,2020-01-03T00:00:00.000Z,True,False,None,3712,EverBank Field,Tennessee,SEC,23.0,"[0, 6, 3, 14]",0.16313978982861882,Indiana,Big Ten,22.0,"[0, 3, 16, 3]",0.8368602101713811
36,401135292,2019,16,postseason,2020-01-03T20:30:00.000Z,True,False,None,3653,Albertsons Stadium,Nevada,Mountain West,21.0,"[3, 6, 0, 12]",0.0020685402629568575,Ohio,Mid-American,30.0,"[3, 17, 10, 0]",0.9979314597370431
37,401135293,2019,16,postseason,2020-01-04T16:30:00.000Z,True,False,None,3616,Amon G. Carter Stadium,Tulane,American Athletic,30.0,"[0, 6, 24, 0]",0.9736480893145654,Southern Mississippi,Conference USA,13.0,"[13, 0, 0, 0]",0.026351910685434632


In [8]:
FBSteams = curSch['home_team'].unique()
FBSteams.sort()

In [9]:
FBSteams

array(['Air Force', 'Akron', 'Alabama', 'Appalachian State', 'Arizona',
       'Arizona State', 'Arkansas', 'Arkansas State', 'Army', 'Auburn',
       'BYU', 'Ball State', 'Baylor', 'Boise State', 'Boston College',
       'Bowling Green', 'Buffalo', 'California', 'Central Michigan',
       'Charlotte', 'Cincinnati', 'Clemson', 'Coastal Carolina',
       'Colorado', 'Colorado State', 'Connecticut', 'Duke',
       'East Carolina', 'Eastern Michigan', 'Florida', 'Florida Atlantic',
       'Florida International', 'Florida State', 'Fresno State',
       'Georgia', 'Georgia Southern', 'Georgia State', 'Georgia Tech',
       "Hawai'i", 'Houston', 'Illinois', 'Indiana', 'Iowa', 'Iowa State',
       'Kansas', 'Kansas State', 'Kent State', 'Kentucky', 'LSU',
       'Liberty', 'Louisiana', 'Louisiana Monroe', 'Louisiana Tech',
       'Louisville', 'Marshall', 'Maryland', 'Memphis', 'Miami',
       'Miami (OH)', 'Michigan', 'Michigan State', 'Middle Tennessee',
       'Minnesota', 'Mississippi St

In [10]:
FBSdict = {}
for team in FBSteams:
    FBSdict[team] = curSch[(curSch['away_team'] == team) | 
                           (curSch['home_team'] == team)]
#     FBSdict[team] = FBSdict[team][np.isfinite(FBSdict[team].away_points)]
    FBSdict[team] = FBSdict[team].reset_index()

In [11]:
FBSbar = tqdm(total=len(FBSteams))
for key in FBSdict:
    FBSbar.set_description(key)
    FBSdict[key]['seasonProg'] = FBSdict[key].apply(lambda row: seasonProgression(row),axis = 1)
    FBSdict[key]['winner'] = FBSdict[key].apply(lambda row: winningTeam(row),axis = 1)
    FBSdict[key]['teamWin'] = FBSdict[key].apply(lambda row: teamWin(row,key),axis = 1)
    
    wins = 0
    losses = 0
    winList = []
    lossList = []
    for row in FBSdict[key].itertuples():
    #     print(row.winner)
        if row.teamWin:
            wins += 1
        else:
            losses += 1
        winList.append(wins)
        lossList.append(losses)
        
    FBSdict[key]['wins'] = winList
    FBSdict[key]['losses'] = lossList
    
    hometeam = FBSdict[key]['home_team'] == key
    FBSdict[key]['home'] = hometeam
    
    FBSdict[key]['winPct'] = FBSdict[key]['wins']/(FBSdict[key]['losses'] + FBSdict[key]['wins'])
    FBSdict[key]['opp'] = FBSdict[key].apply(lambda row: findOpp(row), axis = 1)
    FBSdict[key]['opp_conference'] = FBSdict[key].apply(lambda row: findOppConf(row), axis = 1)
    FBSdict[key]['point_diff'] = FBSdict[key].apply(lambda row: pointDiff(row,key), axis = 1)
    FBSdict[key]['loc_mult'] = FBSdict[key].apply(lambda row: locMult(row), axis = 1)
    FBSdict[key]['conf_mult'] = FBSdict[key].apply(lambda row: confMult(row), axis = 1)
    
    FBSbar.update(1)

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [12]:
FBSbar2 = tqdm(total=len(FBSteams))
for key in FBSdict:
    FBSbar2.set_description(key)
    for w in range(4,curWeek+1):
        FBSdict[key]['win_q_'+str(w)] = FBSdict[key].apply(lambda row: winQ(row, w),axis=1)
    FBSbar2.update()

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: FutureWarning: `item` has been deprecated and will be removed in a future version


In [13]:
FBSdict['Ohio State']

,index,id,season,week,season_type,start_date,neutral_site,conference_game,attendance,venue_id,...,win_q_7,win_q_8,win_q_9,win_q_10,win_q_11,win_q_12,win_q_13,win_q_14,win_q_15,win_q_16
0,26,401112251,2019,1,regular,2019-08-31T16:00:00.000Z,False,False,None,3861,...,0.021270,0.018231,0.019940,0.021270,0.022333,0.022333,0.023203,0.023929,0.024542,0.025068
1,90,401112252,2019,2,regular,2019-09-07T16:00:00.000Z,False,False,None,3861,...,0.045238,0.046531,0.046531,0.047500,0.048254,0.048857,0.049351,0.045238,0.041758,0.042653
2,164,401112159,2019,3,regular,2019-09-14T16:00:00.000Z,False,True,None,3830,...,0.089524,0.095918,0.100714,0.104444,0.104444,0.094000,0.085455,0.089524,0.089524,0.082637
3,247,401112253,2019,4,regular,2019-09-21T19:30:00.000Z,False,False,None,3861,...,0.035913,0.046173,0.053869,0.053869,0.059854,0.064643,0.068561,0.062847,0.066300,0.066300
4,332,401112241,2019,5,regular,2019-09-28T23:30:00.000Z,False,True,None,3834,...,0.115102,0.115102,0.100714,0.089524,0.089524,0.080571,0.091558,0.083929,0.083929,0.083929
5,377,401112216,2019,6,regular,2019-10-05T23:30:00.000Z,False,True,None,3861,...,0.127619,0.127619,0.111667,0.111667,0.099259,0.089333,0.101515,0.111667,0.111667,0.120256
6,446,401112248,2019,8,regular,2019-10-19T00:30:00.000Z,False,True,None,3911,...,0.063857,0.053214,0.045612,0.039911,0.035476,0.063857,0.058052,0.079821,0.079821,0.079821
7,513,401112254,2019,9,regular,2019-10-26T16:00:00.000Z,False,True,None,3861,...,0.335714,0.287755,0.251786,0.251786,0.261111,0.268571,0.274675,0.279762,0.258242,0.239796
8,618,401112209,2019,11,regular,2019-11-09T17:00:00.000Z,False,True,None,3861,...,0.241429,0.206939,0.181071,0.160952,0.144857,0.144857,0.131688,0.120714,0.120714,0.120714
9,681,401112255,2019,12,regular,2019-11-16T20:30:00.000Z,False,True,None,3754,...,0.069643,0.059694,0.104464,0.092857,0.092857,0.083571,0.075974,0.069643,0.069643,0.069643


In [14]:
calcPct('Ohio State',15)

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: FutureWarning: `item` has been deprecated and will be removed in a future version
/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:73: FutureWarning: `item` has been deprecated and will be removed in a future version


0.2666320794590026

In [15]:
FBSbar3 = tqdm(total=len(FBSteams))
FBSpct = {}
for key in FBSdict:
    FBSbar3.set_description(key)
    pctList = []
    for w in range(4,curWeek+1):
        pctList.append(calcPct(key,w))
    FBSpct[key] = pctList
    FBSbar3.update()

/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: FutureWarning: `item` has been deprecated and will be removed in a future version
/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:73: FutureWarning: `item` has been deprecated and will be removed in a future version


In [16]:
PCTdf = pd.DataFrame.from_dict(FBSpct,orient='index',columns=range(4,curWeek+1))
PCTdf.to_csv('2019/PCT.csv')

In [17]:
PCTnormdf = pd.DataFrame(index = FBSteams,columns=range(4,curWeek+1))

for w in range(4,curWeek+1):
    x = PCTdf[w].values.reshape(-1,1)
    min_max_scaler = preprocessing.MinMaxScaler()
    xScaled = min_max_scaler.fit_transform(x)
    PCTnormdf[w] = xScaled.reshape(130)

PCTnormdf.to_csv('2019/PCT_norm.csv')

In [18]:
Ranks = PCTdf.rank(method='first',ascending=False).astype('int64')
Ranks.to_csv('2019/Ranks.csv')

In [19]:
for w in range(4,curWeek+1):
    Wdf = pd.concat([PCTnormdf[w],Ranks[w]],axis=1)
    Wdf.columns = ['PCT','Rank']
    Wdf = Wdf.sort_values('Rank')
    Wdf.to_csv('2019/W%s.csv'%w)

In [22]:
Ranks.sort_values(16).head(30)

,4,5,6,7,8,9,10,11,12,13,14,15,16
LSU,15,14,12,4,2,4,4,1,2,3,2,2,1
Clemson,2,3,5,8,6,1,1,2,1,2,3,3,2
Ohio State,1,2,1,2,4,2,3,3,3,1,1,1,3
Georgia,6,9,10,24,18,19,11,7,4,4,4,6,4
Oregon,27,35,16,10,9,8,6,6,5,14,10,5,5
Oklahoma,25,10,8,6,5,11,12,10,8,6,5,4,6
Notre Dame,37,13,18,14,15,29,19,13,13,9,8,8,7
Appalachian State,16,12,20,15,13,12,22,17,15,16,14,11,8
Florida,8,11,2,12,10,13,21,16,14,19,12,15,9
Penn State,17,5,3,3,1,3,2,8,6,12,11,13,10
